<a href="https://colab.research.google.com/github/jsKim-prog/pythonStudy23_JS/blob/master/Colab_4%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas-datareader finance-datareader

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr

In [ ]:
def get_return(code, n):
  df = fdr.DataReader(code,'2018')

  df = df[['Close']]

  df['ma'] = df.rolling(n).mean().shift(1)

  df['action'] = np.where(df['Close'] > df['ma'], 'Buy', 'Sell')

  df.iloc[-1,-1] = 'Sell'

  cond1 = (df['action'] == 'Buy')&(df['action'].shift(1) == 'Sell')
  cond2 = (df['action'] == 'Sell')&(df['action'].shift(1) == 'Buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜', '종가(buy)', '이평값','액션']

  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜', '종가(sell)', '이평값','액션']

  df_result = pd.concat([df_buy,df_sell],axis=1)
  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']

  return df_result[['수익률']].cumprod().iloc[-1,-1]-1

In [ ]:
def get_return_sl(code, short, long):
  df = fdr.DataReader(code,'2018')

  df = df[['Close']].copy()

  df['ma1'] = df['Close'].rolling(short).mean().shift(1)
  df['ma2'] = df['Close'].rolling(long).mean().shift(1)

  df['action'] = np.where(df['ma1'] > df['ma2'], 'Buy', 'Sell')

  df.iloc[-1,-1] = 'Sell'

  cond1 = (df['action'] == 'Buy')&(df['action'].shift(1) == 'Sell')
  cond2 = (df['action'] == 'Sell')&(df['action'].shift(1) == 'Buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜', '종가(buy)', '이평값1','이평값2','액션']

  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜', '종가(sell)','이평값1','이평값2','액션']

  df_result = pd.concat([df_buy,df_sell],axis=1)
  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']

  df_final = (df_result[['수익률']].cumprod().tail(1) -1)*100
  df_final['단기'] = short
  df_final['장기'] = long

  return df_final

In [ ]:
dfs = []
for short in range(3,11):
  for long in range(30,61):
    df = get_return_sl('086790', short, long)
    dfs.append(df)

df_result = pd.concat(dfs)
df_result.sort_values(by='수익률',ascending=False).head(3)

,수익률,단기,장기
24,29.426561,10,56
23,19.799474,10,55
25,19.541900,10,59
